In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector

#### importing and loading data

In [4]:
%cd ..

/Users/Marie/code/MarieMCodes/re_value_predictor


/Users/Marie/.pyenv/versions/3.10.6/envs/re_value_predictor/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [48]:
data=pd.read_csv('raw_data/london_re_postcodes_latlon_master.zip',
                    dtype={'price': np.int32,'month':np.int8,'year':np.int16},
                    ).sample(100000)
    # master: drop month, drop date
data.drop(columns=['date','month'],inplace=True)

In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 408495 to 1061438
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   price          100000 non-null  int32  
 1   property_type  100000 non-null  object 
 2   property_age   100000 non-null  object 
 3   ownership      100000 non-null  object 
 4   year           100000 non-null  int16  
 5   lat            100000 non-null  float64
 6   lon            100000 non-null  float64
 7   sin_time       100000 non-null  float64
 8   cos_time       100000 non-null  float64
dtypes: float64(4), int16(1), int32(1), object(3)
memory usage: 6.7+ MB


In [50]:
X=data.drop(columns='price')
y=data['price']
X

,property_type,property_age,ownership,year,lat,lon,sin_time,cos_time
408495,F,N,L,2005,51.607175,-0.238842,-1.000000e+00,-1.836970e-16
1108372,F,N,L,2014,51.593582,-0.003255,-5.000000e-01,8.660254e-01
505234,T,N,F,2006,51.445045,-0.045705,1.224647e-16,-1.000000e+00
1965305,F,N,L,2022,51.502128,-0.162967,-8.660254e-01,-5.000000e-01
1185078,F,N,L,2014,51.440542,-0.125136,-2.449294e-16,1.000000e+00
...,...,...,...,...,...,...,...,...
630929,D,N,F,2007,51.347421,-0.171085,-5.000000e-01,-8.660254e-01
38200,D,N,F,1998,51.585333,-0.234270,-8.660254e-01,-5.000000e-01
569310,S,N,F,2006,51.372043,-0.035580,5.000000e-01,8.660254e-01
855530,F,Y,L,2010,51.442130,-0.187180,5.000000e-01,8.660254e-01


### Preprocessing data

In [56]:
X.columns = X.columns.str.strip()
print(X.columns)

# Define numerical and categorical columns
numeric_columns = ['year','sin_time','cos_time'] # ,'lat','lon']   keeping those as is and not scaling them
categorical_columns = ['property_type', 'property_age', 'ownership']

# Preprocessing pipelines for numerical and categorical data
numeric_pipeline = Pipeline([
#('imputer', SimpleImputer(strategy='median')),
        ('scaler', RobustScaler())
    ])
numeric_pipeline

Index(['property_type', 'property_age', 'ownership', 'year', 'lat', 'lon',
       'sin_time', 'cos_time'],
      dtype='object')


Pipeline(steps=[('scaler', RobustScaler())])

In [57]:
categorical_pipeline = Pipeline([
        #('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore',
                                 drop='if_binary',
                                 sparse_output=False
                                 ).set_output(transform='pandas')
        )
    ])
categorical_pipeline

Pipeline(steps=[('onehot',
                 OneHotEncoder(drop='if_binary', handle_unknown='ignore',
                               sparse_output=False))])

In [58]:
# Column transformer to preprocess both numerical and categorical data
preprocessor = ColumnTransformer([
        ('num', numeric_pipeline, numeric_columns),
        ('cat', categorical_pipeline, make_column_selector(dtype_include="object"))
    ], remainder='passthrough') # no need to scale lat lon
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num',
                                 Pipeline(steps=[('scaler', RobustScaler())]),
                                 ['year', 'sin_time', 'cos_time']),
                                ('cat',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x12bb8c520>)])

In [54]:
X

,property_type,property_age,ownership,year,lat,lon,sin_time,cos_time
408495,F,N,L,2005,51.607175,-0.238842,-1.000000e+00,-1.836970e-16
1108372,F,N,L,2014,51.593582,-0.003255,-5.000000e-01,8.660254e-01
505234,T,N,F,2006,51.445045,-0.045705,1.224647e-16,-1.000000e+00
1965305,F,N,L,2022,51.502128,-0.162967,-8.660254e-01,-5.000000e-01
1185078,F,N,L,2014,51.440542,-0.125136,-2.449294e-16,1.000000e+00
...,...,...,...,...,...,...,...,...
630929,D,N,F,2007,51.347421,-0.171085,-5.000000e-01,-8.660254e-01
38200,D,N,F,1998,51.585333,-0.234270,-8.660254e-01,-5.000000e-01
569310,S,N,F,2006,51.372043,-0.035580,5.000000e-01,8.660254e-01
855530,F,Y,L,2010,51.442130,-0.187180,5.000000e-01,8.660254e-01


In [59]:
X_transformed = preprocessor.fit_transform(X)

In [60]:
X_transformed.shape

(100000, 11)

In [ ]:
dislay(pd.DataFrame(
    preprocessed_pipeline,
    columns=preprocessor.get_feature_names_out()

In [61]:
X_transformed

array([[-6.36363636e-01, -7.32050808e-01,  0.00000000e+00, ...,
         1.00000000e+00,  5.16071750e+01, -2.38842000e-01],
       [ 1.81818182e-01, -3.66025404e-01,  6.33974596e-01, ...,
         1.00000000e+00,  5.15935820e+01, -3.25500000e-03],
       [-5.45454545e-01,  2.68951103e-16, -7.32050808e-01, ...,
         0.00000000e+00,  5.14450450e+01, -4.57050000e-02],
       ...,
       [-5.45454545e-01,  3.66025404e-01,  6.33974596e-01, ...,
         0.00000000e+00,  5.13720430e+01, -3.55800000e-02],
       [-1.81818182e-01,  3.66025404e-01,  6.33974596e-01, ...,
         1.00000000e+00,  5.14421300e+01, -1.87180000e-01],
       [ 9.09090909e-02,  0.00000000e+00,  7.32050808e-01, ...,
         0.00000000e+00,  5.15878150e+01, -3.39682000e-01]])